In [13]:
import datetime as dt
from odds_and_other_projections import *
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support.ui import Select
from webdriver_manager.chrome import ChromeDriverManager

yesterday_string = '20220425'
yesterdays_capital_538 = 100000
team_map = {
    'Giants' : 'San Francisco Giants',
    'Astros' : 'Houston Astros',
    'Brewers': 'Milwaukee Brewers',
    'Dodgers': 'Los Angeles Dodgers',
    'Rays' : 'Tampa Bay Rays',
    'Red Sox': 'Boston Red Sox',
    'White Sox': 'Chicago White Sox',
    'Padres': 'San Diego Padres',
    'Athletics': 'Oakland Athletics',
    'Yankees' : 'New York Yankees',
    'Mariners': 'Seattle Mariners',
    'Reds': 'Cincinnati Reds',
    'Blue Jays' : 'Toronto Blue Jays',
    'Mets' : 'New York Mets',
    'Phillies': 'Philadelphia Phillies',
    'Angels': 'Los Angeles Angels',
    'Braves': 'Atlanta Braves',
    'Cardinals': 'St. Louis Cardinals',
    'Tigers': 'Detroit Tigers',
    'Cubs': 'Chicago Cubs',
    'Indians': 'Cleveland Indians',
    'Guardians' : 'Cleveland Guardians',
    'Nationals': 'Washington Nationals',
    'Rockies': 'Colorado Rockies',
    'Marlins': 'Miami Marlins',
    'Royals': 'Kansas City Royals',
    'Twins': 'Minnesota Twins',
    'Pirates': 'Pittsburgh Pirates',
    'Rangers': 'Texas Rangers',
    'Orioles': 'Baltimore Orioles',
    'Diamondbacks': 'Arizona Diamondbacks',
    'D-backs' : 'Arizona Diamondbacks'
}

In [14]:
# Function to calculate potential payoffs
def calculate_payoff_external(row):
    if row.Bet_538<=0:
        payoff = 0
    else:
        if row.Home_KC_538>0:
            if row.Home_Odds>0:
                payoff = (row.Home_Odds/100)*row.Bet_538
            if row.Home_Odds<0:
                payoff = row.Bet_538/((abs(row.Home_Odds)/100))
        if row.Away_KC_538>0:
            if row.Away_Odds>0:
                payoff = (row.Away_Odds/100)*row.Bet_538
            if row.Away_Odds<0:
                payoff = row.Bet_538/((abs(row.Away_Odds)/100))
    return payoff

# Getting yesterday's results from CBS
link = 'https://www.cbssports.com/mlb/scoreboard/' + yesterday_string + '/'
tables = pd.read_html(link)
results_table = pd.DataFrame(columns = ['Home_Team', 'Away_Team', 'Winner'])
for table in tables:
    if list(table.columns) == ['Unnamed: 0', 'R', 'H', 'E']:
        # Getting team names
        team_away_list = table.iloc[0,0].split(' ')
        del team_away_list[-2:]
        if len(team_away_list) == 2:
            team_away = team_away_list[0] + ' ' + team_away_list[1]
        else:
            team_away = team_away_list[0]
        team_home_list = table.iloc[1,0].split(' ')
        del team_home_list[-2:]
        if len(team_home_list) == 2:
            team_home = team_home_list[0] + ' ' + team_home_list[1]
        else:
            team_home = team_home_list[0]
        # Getting score and determining winner
        runs_away = table.iloc[0,1]
        runs_home = table.iloc[1,1]
        if runs_away>runs_home:
            winner = team_away
        else:
            winner = team_home
        # Appending to results table
        series = pd.Series([team_home, team_away, winner], index = results_table.columns)
        results_table = results_table.append(series, ignore_index = True)        
    else:
        continue
for column in list(results_table.columns):
    results_table[column] = results_table[column].apply(lambda x: team_map[x])

# Reading in yesterdays bets and creating tracker columns
yesterdays_bets = pd.read_csv('past_bets/external/bets_' + yesterday_string + '.csv', index_col = 0)
yesterdays_bets = yesterdays_bets[(yesterdays_bets.Home_Team != '538') & (yesterdays_bets.Away_Team != '538')]
yesterdays_bets['Home_Team'] = yesterdays_bets['Home_Team'].apply(lambda x: team_map[x])
yesterdays_bets['Away_Team'] = yesterdays_bets['Away_Team'].apply(lambda x: team_map[x])
yesterdays_bets = yesterdays_bets[yesterdays_bets.Bet_538>0]
yesterdays_bets.reset_index(drop = True, inplace = True)
yesterdays_bets['Won_538'] = 0
yesterdays_bets['Tracker_538'] = 0
for index, row in yesterdays_bets.iterrows():
    home_team = row.Home_Team
    away_team = row.Away_Team
    payoff_538 = calculate_payoff_external(row)
    if row.Bet_538>0:
        if (row.Home_KC_538>0) & (home_team in results_table['Winner'].values):
            yesterdays_bets.loc[index, 'Won_538'] = 1
        elif (row.Away_KC_538>0) & (away_team in results_table['Winner'].values):
            yesterdays_bets.loc[index, 'Won_538'] = 1
        else:
            yesterdays_bets.loc[index, 'Won_538'] = 0
    else:
        yesterdays_bets.loc[index, 'Won_538'] = -1
    if yesterdays_bets.loc[index, 'Won_538'] == 1:
        if index == 0:
            yesterdays_bets.loc[index, 'Tracker_538'] = yesterdays_capital_538 + payoff_538
        else:
            yesterdays_bets.loc[index, 'Tracker_538'] = yesterdays_bets.loc[(index-1), 'Tracker_538'] + payoff_538
    else:
        if index == 0:
            yesterdays_bets.loc[index, 'Tracker_538'] = yesterdays_capital_538 - row.Bet_538
        else:
            yesterdays_bets.loc[index, 'Tracker_538'] = yesterdays_bets.loc[(index-1), 'Tracker_538'] - row.Bet_538
        
# return yesterdays_bets

In [15]:
yesterdays_bets

,Date,Away_Team,Home_Team,Away_Prob_538,Home_Prob_538,Home_Odds,Away_Odds,Home_Prob_Odds,Away_Prob_Odds,Home_KC_538,Away_KC_538,Bet_538,Won_538,Tracker_538
0,1900-04-25,Colorado Rockies,Philadelphia Phillies,0.41,0.59,-200.0,168.0,0.666667,0.373134,0.000000,0.005881,588.095238,0,99411.904762
1,1900-04-25,New York Mets,St. Louis Cardinals,0.53,0.47,126.0,-148.0,0.442478,0.596774,0.004937,0.000000,493.650794,0,98918.253968
